In [2]:
import pandas as pd
import numpy as np
import requests
import time

In [3]:
df = pd.read_csv("../data/processed/cnpjs_amostra.csv", delimiter = ',')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CNPJ           100 non-null    int64 
 1   data_abertura  80 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [5]:
def consulta_cnpj(cnpj):
    cnpj = ''.join(filter(str.isdigit, str(cnpj)))
    url = f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}"
    
    try:
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            data = response.json().get("data_inicio_atividade")
            return data
        
        elif response.status_code == 404:
            print(f"CNPJ não encontrado: {cnpj}")
            return None
        
        elif response.status_code == 429:
            print("Rate limit atingido. Aguardando 5 segundos...")
            time.sleep(5)
            return consulta_cnpj(cnpj)
        
        else:
            print(f"Erro {response.status_code} para {cnpj}")
            return None

    except Exception as e:
        print(f"Erro na requisição: {e}")
        return None

for i, row in df[df["data_abertura"].isna()].iterrows():
    print(f"Consultando {row['CNPJ']}...")
    
    data_api = consulta_cnpj(row["CNPJ"])
    
    if data_api:
        df.at[i, "data_abertura"] = data_api
    
    time.sleep(1)  

df.to_csv("../data/processed/cnpjs_preenchido.csv", index=False)

print("Processo finalizado.")

Consultando 26765666000124...
Consultando 91646638000150...
Consultando 38022734000144...
Consultando 22978240000106...
Consultando 19691435000158...
Consultando 24276137000131...
Consultando 3977536000400...
Erro 400 para 3977536000400
Consultando 37657544000130...
Consultando 6028254000102...
Erro 400 para 6028254000102
Consultando 31273722000143...
Consultando 36080417000159...
Consultando 22079365000196...
Consultando 20750903000105...
Consultando 7210925008271...
Erro 400 para 7210925008271
Consultando 3574734000172...
Erro 400 para 3574734000172
Consultando 11204448000143...
Consultando 28792763000131...
Consultando 5656085000184...
Erro 400 para 5656085000184
Consultando 43463378000134...
Consultando 16647289000337...
Processo finalizado.


Aqui, em um sistema de empresa, acredito que não vá ocasionar erros, o BRASILAPI é grátis, mas acredito que limitado nas pesquisas e não totalmente atualizado, diferente do serasa por exemplo. 

In [7]:
df2 = pd.read_csv('../data/processed/cnpjs_preenchido.csv', delimiter=',')

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CNPJ           100 non-null    int64 
 1   data_abertura  95 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
